In [ ]:
import pandas as pd

In [ ]:
has_ans= pd.read_csv('has_answer_newsqa.csv')
has_ans= has_ans.drop(has_ans.columns[0], axis=1)
has_ans= has_ans.drop(has_ans.columns[6], axis=1)
has_ans['possible_answer'] = [x[1:-1] for x in has_ans['possible_answer']]

In [ ]:

has_ans.shape

In [ ]:
has_ans['answer_text_in_prediction'] = has_ans.apply(lambda x: x.pred_answer in x.possible_answer, axis=1)
has_ans['answer_prediction_in_answer_text'] = has_ans.apply(lambda x: x.possible_answer in x.pred_answer, axis=1)
has_ans['TF_F1']= has_ans.answer_prediction_in_answer_text+has_ans.answer_text_in_prediction # logical OR operator
has_ans= has_ans.drop(has_ans[['answer_text_in_prediction','answer_prediction_in_answer_text']], axis=1)

In [ ]:
has_ans ['question']= has_ans ['question'].str.replace('[^\w\s]','')
has_ans ['question']= has_ans ['question'].str.lower()


In [ ]:
has_ans = has_ans[has_ans['TF'].notna()]


In [ ]:
has_ans['question_type']=''
keywords = ["why", "when", "how", "what", "who", "where",'which']
for  (i, answer) in enumerate(has_ans['question']):
    words = answer.split()
    for key in keywords:
        if key in  words:
            has_ans['question_type'][i]=key

In [ ]:
has_ans['question_type']=has_ans['question_type'].replace('', 'other')


In [ ]:
has_ans.question_type.value_counts()

In [ ]:
has_ans['question_type']=has_ans['question_type'].replace('', 'other')

In [ ]:
has_ans.question_type.value_counts()

In [ ]:
has_ans.head()

<ul>
  <li>context - provided context</li>
  <li>id - ID of question</li>
  <li>TF - whether the prediction was correct or not (True/False)</li>
  <li>pred_answer - what was the predicted answer</li>
  <li>possible_answers - acceptable answers</li>
  <li>question - question ansked</li>
  <li>TF_F1 - approx. correct answers according to F1 score</li>

</ul>

Corresponds with the scores: 
<ul>
    <li> "exact": 53.56174990321332,</li>
    <li>"f1": 69.15141871385678, </li>
    <li>"total": 5166 , </li>
    <li>"HasAns_exact": 53.56174990321332, </li>
    <li>"HasAns_f1": 69.15141871385678, </li>
    <li>"HasAns_total": 5166,</li>
    <li>"best_exact": 53.56174990321332,</li>
    <li>"best_exact_thresh": -11.604235649108887, </li>
    <li>"best_f1":  69.15141871385688, </li>
    <li>"best_f1_thresh": -8.893298149108887</li>
 </ul>

In [ ]:
has_ans.TF_F1.value_counts()

In [ ]:
has_ans.TF.value_counts()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
def plot_histogram(data, label='',datalabel=''):
    plt.hist(data, bins=100, normed=1, alpha=1.0)
    plt.xlabel('Lengths')
    plt.ylabel('Probability')
    plt.grid(True)
    plt.title((datalabel+' '+label+' length').capitalize())
    plt.savefig('graphs/'+datalabel+'_length_'+label+'.png')
    plt.show()
    plt.clf()

In [ ]:
plot_histogram([(len(str(a))) for a in has_ans['pred_answer']], 'answer','has_ans')
plot_histogram([q for q in has_ans['question'].str.len()], 'question','has_ans')
plot_histogram([c for c in has_ans['context'].str.len()], 'context','has_ans')

In [ ]:
has_ans['ans_length']  = has_ans['pred_answer'].str.len()
has_ans['con_length']  = has_ans['context'].str.len()
has_ans['que_length']  = has_ans['question'].str.len()

In [ ]:
def question_questiontype(dataset,label,x,datalabel):
    answers= []
    keywords = ["why", "when", "how", "what", "who", "where",'which','other']
    for key in keywords:
        answers.append([len(answer) for (i, answer) in enumerate(dataset) if key in has_ans.iloc[i]['question_type'].lower()])
    bins = np.linspace(0, x, 15)
    plt.hist(answers, bins, normed=True, label=keywords)
    plt.xlabel('Lengths')
    plt.ylabel('Probability')
    plt.grid(True)
    plt.legend()
    plt.title((datalabel+' '+label+' questiontype').capitalize())
    plt.savefig('graphs/'+datalabel+'_questiontype_'+label+'.png')
    plt.show()
    plt.clf()

In [ ]:
question_questiontype(has_ans['pred_answer'],'answer',100,'has_ans' )
question_questiontype(has_ans['context'],'context',3000,'has_ans')
question_questiontype(has_ans['question'],'question',150,'has_ans')

In [ ]:
incorrect=has_ans[has_ans['TF'] == False]
correct=has_ans[has_ans['TF']==True]

# Only incorrect answers

In [ ]:
plot_histogram([a for a in incorrect['pred_answer'].str.len()], 'answer','has_ans_incorrect')
plot_histogram([q for q in incorrect['question'].str.len()], 'question','has_ans_incorrect')
plot_histogram([c for c in incorrect['context'].str.len()], 'context','has_ans_incorrect')

In [ ]:
question_questiontype(incorrect['pred_answer'],'answer',100,'has_ans_incorrect' )
question_questiontype(incorrect['context'],'context',3000,'has_ans_incorrect')
question_questiontype(incorrect['question'],'question',150,'has_ans_incorrect')

# Only correct answers


In [ ]:
plot_histogram([a for a in correct['pred_answer'].str.len()], 'answer','has_ans_correct')
plot_histogram([q for q in correct['question'].str.len()], 'question','has_ans_correct')
plot_histogram([c for c in correct['context'].str.len()], 'context','has_ans_correct')

In [ ]:
question_questiontype(correct['pred_answer'],'answer',100,'has_ans_correct' )
question_questiontype(correct['context'],'context',3000,'has_ans_correct')
question_questiontype(correct['question'],'question',150,'has_ans_correct')

In [ ]:
sns.histplot(data=has_ans, x="ans_length",bins=100, hue='TF',palette=["C0", "C1"])
plt.show()

## Pie charts


In [ ]:

def piechart (dataset,labels,name):
    values=dataset.value_counts()
    labels=dataset.value_counts().index.values
    plt.figure(figsize=(4,4))
    plt.pie(values,labels=labels,autopct='%1.1f%%',shadow=True)
    plt.legend(labels)
    plt.axis('equal')
    plt.tight_layout()
    plt.title((name+' question type distribution').capitalize())
    plt.savefig('graphs/'+name+'_questiontype_pie.png')
    plt.show()
    print(dataset.value_counts())



In [ ]:
labels=''
piechart(has_ans['question_type'],labels,'has_ans')
piechart(incorrect['question_type'],labels,'has_ans_incorrect')
piechart(correct['question_type'],labels,'has_ans_correct')

# With the F1 score

In [ ]:
incorrect=has_ans[has_ans['TF_F1'] == False]
correct=has_ans[has_ans['TF_F1']==True]

# Only incorrect answers

In [ ]:
plot_histogram([a for a in incorrect['pred_answer'].str.len()], 'answer','has_ans_incorrect')
plot_histogram([q for q in incorrect['question'].str.len()], 'question','has_ans_incorrect')
plot_histogram([c for c in incorrect['context'].str.len()], 'context','has_ans_incorrect')

In [ ]:
question_questiontype(incorrect['pred_answer'],'answer',100,'has_ans_incorrect' )
question_questiontype(incorrect['context'],'context',3000,'has_ans_incorrect')
question_questiontype(incorrect['question'],'question',150,'has_ans_incorrect')

# Only correct answers


In [ ]:
plot_histogram([a for a in correct['pred_answer'].str.len()], 'answer','has_ans_correct')
plot_histogram([q for q in correct['question'].str.len()], 'question','has_ans_correct')
plot_histogram([c for c in correct['context'].str.len()], 'context','has_ans_correct')

In [ ]:
question_questiontype(correct['pred_answer'],'answer',100,'has_ans_correct' )
question_questiontype(correct['context'],'context',3000,'has_ans_correct')
question_questiontype(correct['question'],'question',150,'has_ans_correct')

In [ ]:
sns.histplot(data=has_ans, x="ans_length",bins=100, hue='TF',palette=["C0", "C1"])
plt.show()

## Pie charts


In [ ]:
labels=''
piechart(has_ans['question_type'],labels,'has_ans')
piechart(incorrect['question_type'],labels,'has_ans_incorrect')
piechart(correct['question_type'],labels,'has_ans_correct')

In [ ]:
# crosstab
df=has_ans
pal = ["red","royalblue" ]
ax= pd.crosstab(df['question_type'],df['TF'] ).apply(lambda r: r/r.sum()*100, axis=1)
ax=ax.sort_values([1], ascending=True)
ax_1 = ax.plot.bar(figsize=(10,6),stacked=True, rot=0,color=pal, sort_columns=True)
display(ax)

plt.legend(loc='upper center', bbox_to_anchor=(1.0,1.0), title="Correct Prediction")

plt.xlabel('Question type')
plt.ylabel('Percent Distribution')

for rec in ax_1.patches:
    height = rec.get_height()
    ax_1.text(rec.get_x() + rec.get_width() / 2, 
              rec.get_y() + height / 2,
              "{:.0f}%".format(height),
              ha='center', 
              va='bottom')
plt.savefig('graphs/NewsQA_questiontype_distribution.png')
plt.show()

In [ ]:

how_analysis=has_ans

In [ ]:
how_analysis=how_analysis.reset_index(drop=True)
how_analysis['question']=how_analysis['question'].str.lower()


In [ ]:
how_analysis.head()


In [ ]:
how_analysis['question_type']=''
qty=[]
keywords = ['how','far','long','many', 'much','old', 'come']
for index, row in how_analysis.iterrows():

    answer=row['question']
    words = answer.split()
    for key in keywords:
        if key in  words:
            qty.append(key)
    how_analysis['question_type'][index]=qty
    qty=[]



In [ ]:
how_analysis=how_analysis[how_analysis.question_type.apply(lambda x: len(x) > 1)]

In [ ]:
how_analysis.question_type.value_counts()

In [ ]:
how_analysis=how_analysis[how_analysis['question_type']!='']

In [ ]:
how_analysis['question_type'] = how_analysis['question_type'].str[1]

In [ ]:
gh=how_analysis[how_analysis['question_type']=='many']
gh.sample(6)

In [ ]:
# crosstab
df=how_analysis
pal = ["red","royalblue" ]
ax= pd.crosstab(df['question_type'],df['TF'] ).apply(lambda r: r/r.sum()*100, axis=1)
ax=ax.sort_values([1], ascending=True)
ax_1 = ax.plot.bar(figsize=(10,6),stacked=True, rot=0,color=pal, sort_columns=True)
display(ax)

plt.legend(loc='upper center', bbox_to_anchor=(1.0, 1.0), title="Correct Prediction")

plt.xlabel('Question type')
plt.ylabel('Percent Distribution')

for rec in ax_1.patches:
    height = rec.get_height()
    ax_1.text(rec.get_x() + rec.get_width() / 2, 
              rec.get_y() + height / 2,
              "{:.0f}%".format(height),
              ha='center', 
              va='bottom')
plt.savefig('graphs/how_questiontype_distribution.png')
plt.show()

In [ ]:
has_ans[has_ans['question_type']=='other']

# ERROR ANALYSIS

# predicted answer is longer than the original but still conveys correct information

In [ ]:
inc=has_ans[has_ans['TF'] == False]

In [ ]:
inc['answer_text_in_prediction'] = inc.apply(lambda x: x.possible_answer in x.pred_answer, axis=1)

In [ ]:
inc.head()

In [ ]:
inc['possible_answer_length']  = inc['possible_answer'].str.len()
inc['difference'] = inc.apply(lambda x: x.possible_answer_length < x.ans_length, axis=1)

In [ ]:
inc['kkt'] = inc.apply(lambda x: x.answer_text_in_prediction &  x.difference, axis=1)

In [ ]:
inc.kkt.value_counts()

# checking whether the predicted answer is shorter than the original answer but still conveys same information

In [ ]:
inc['answer_prediction_in_answer_text'] = inc.apply(lambda x: x.pred_answer in x.possible_answer, axis=1)

In [ ]:
inc['difference2'] = inc.apply(lambda x: x.possible_answer_length > x.ans_length, axis=1)

In [ ]:
inc['kkt2'] = inc.apply(lambda x: x.answer_prediction_in_answer_text &  x.difference2, axis=1)

In [ ]:
inc.kkt2.value_counts()

In [ ]:
pd.set_option('display.max_colwidth', None)

# getting examples

In [ ]:

#will result in error since tehre are none longer and shorter at the same time

#example=inc[(inc['kkt'] == True)& (inc['kkt2'] == True)]  
#example.sample()

In [ ]:
example.kkt.value_counts()

In [ ]:
inc=inc[((inc['kkt'] & inc['kkt2']) == False)]  

In [ ]:
#longer
longer=inc[~(inc['kkt'] == False)]  


In [ ]:
longer.kkt.value_counts()

In [ ]:
longer.sample()

In [ ]:
piechart(longer['question_type'],labels,'')

In [ ]:
#shorter
shorter=inc[~(inc['kkt2'] == False)]  

In [ ]:
shorter.kkt2.value_counts()

In [ ]:
shorter.sample()

In [ ]:
piechart(shorter['question_type'],labels,'')

In [ ]:
#wrong predictions
wrong=inc[~(inc['kkt2'] == True )& ~(inc['kkt'] == True )]  

In [ ]:
wrong.sample()

In [ ]:
wrong.sample()

In [ ]:
piechart(wrong['question_type'],labels,'')

# analysis of char-count

In [ ]:
answ=has_ans
def analyse_lowest(part, label):
    se=100000
    for q in part.str.len():
        if se>q:
            se=q
            
    print('The length of ',label,'starts at ',se)
def analyse_highest(part, label):
    se=0
    for q in part.str.len():
        if se<q:
            se=q
            
    print('The length of ',label,'ends at ',se)
answ = answ[answ['possible_answer'].notna()]



In [ ]:
analyse_lowest(answ['possible_answer'],'original answer')
analyse_lowest(answ['context'],'context')
analyse_lowest(answ['question'],'question')
analyse_lowest(answ['pred_answer'],'pred_answer')
analyse_highest(answ['possible_answer'],'original answer')
analyse_highest(answ['context'],'context')
analyse_highest(answ['question'],'question')
analyse_highest(answ['pred_answer'],'pred_answer')

In [ ]:
#allquestiontype pairs 
has_ans['question_type']=''
qty=[]
keywords = ["why", "when", "how", "what", "who", "where",'which', 'whom','whose','does']
for index, row in has_ans.iterrows():

    answer=row['question']
    words = answer.split()
    for key in keywords:
        if key in  words:
            qty.append(key)
    has_ans['question_type'][index]=qty
    qty=[]
has_ans.question_type.value_counts()


# exports for manual analysis of the wrong answers

In [ ]:
export=wrong[['pred_answer','possible_answer','question','id']]

In [ ]:
export.to_json('kek.json',orient="records")